In [1]:
import ultralytics
from ultralytics import YOLO
ultralytics.checks()
import wandb
import random  # for demo script
import os
import argparse

from ultralytics import YOLO
from ultralytics import settings

Ultralytics YOLOv8.1.7 🚀 Python-3.11.7 torch-2.1.2+cu121 CUDA:0 (NVIDIA RTX A5000, 24040MiB)
Setup complete ✅ (11 CPUs, 116.4 GB RAM, 13.5/64.8 GB disk)


In [2]:
os.chdir("/work/csr33/object_detection")
# Update settings
# https://docs.ultralytics.com/quickstart/
os.makedirs('/work/csr33/object_detection/weights', exist_ok=True)
settings.update({'weights_dir': '/work/csr33/object_detection/weights'})

os.makedirs('/work/csr33/object_detection/runs', exist_ok=True)
settings.update({'runs_dir': '/work/csr33/object_detection/runs'})

settings.update({'neptune': False, 'clearml': False, 
                 "raytune": False, #'comet': True, 'raytune': True,
                 'dvc': False, 'hub': False, 'mlflow': False,
                 'tensorboard': False, 'wandb': True})

In [13]:
#Set up wandb
WANDB_API_KEY="006286beb62f8eedc5994d45a84434bf72b09040"
# Name and notes optional
WANDB_NAME="My first run"
WANDB_NOTES="Smaller learning rate, more regularization."
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /hpc/home/csr33/.netrc


True

In [3]:
def get_args_parse():
    parser = argparse.ArgumentParser("Exploratory Data Analysis")
    parser.add_argument("--data", default="/hpc/home/csr33/ast_object_detection/ast.yaml", type=str)
    parser.add_argument("--model", default='yolov8n.pt', type=str)
    parser.add_argument("--batch", default=8, type=int)
    parser.add_argument("--imgsz", default=640, type=int)
    parser.add_argument("--epochs", default=2, type=int)
    args = parser.parse_args()
    return args

In [4]:
import sys
sys.argv = ['my_notebook']
args = get_args_parse()

In [5]:
best_hyper_parameter_yaml = "/work/csr33/object_detection/runs/detect/tune3/best_hyperparameters.yaml"

In [6]:
# Load a model
model = YOLO('yolov8x.pt')  # load a pretrained model (recommended for training)

In [ ]:
results = model.train(
   data=args.data,
   imgsz=args.imgsz,
   epochs=args.epochs,
   batch=args.batch,
    workers=4,
    cfg = best_hyper_parameter_yaml,
   name=f'{os.path.splitext(args.model)[0]}_e{args.epochs}_b{args.batch}_imgsz{args.imgsz}')


New https://pypi.org/project/ultralytics/8.1.9 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.7 🚀 Python-3.11.7 torch-2.1.2+cu121 CUDA:0 (NVIDIA RTX A5000, 24040MiB)
engine/trainer: task=detect, mode=train, model=None, data=/hpc/home/csr33/ast_object_detection/ast.yaml, epochs=2, time=None, patience=50, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=4, project=None, name=yolov8n_e2_b8_imgsz6402, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: csr33 (turn). Use `wandb login --relogin` to force relogin


Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


train: Scanning /work/csr33/ast/datasets/ast_yolov5/labels/train.cache... 19347 images, 0 backgrounds, 0 corrupt: 100%|██████████| 19347/19347 [00:00<?, ?it/s]

train: WARNING ⚠️ /work/csr33/ast/datasets/ast_yolov5/images/train/m_3812263_se_10_060_20180723_15_18.jpg: 1 duplicate labels removed


...


/hpc/group/borsuklab/cred/.conda/envs/yolov8_env/lib/python3.11/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 6, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Plotting labels to /work/csr33/object_detection/runs/detect/yolov8n_e2_b8_imgsz6402/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000909, momentum=0.9) with parameter groups 97 weight(decay=0.0), 104 weight(decay=0.0005), 103 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 4 dataloader workers
Logging results to /work/csr33/object_detection/runs/detect/yolov8n_e2_b8_imgsz6402
Starting training for 2 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/2      7.27G      1.038      1.074          1         44        640: 100%|██████████| 2419/2419 [10:06<00:00,  3.99it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 260/260 [00:49<00:00,  5.30it/s]


                   all       4146      21256      0.652      0.538      0.522      0.358

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        2/2      7.58G     0.9844     0.8512     0.9852         34        640:  16%|█▌        | 378/2419 [01:29<07:43,  4.40it/s]

In [ ]:
# Use the model
results = model.val()  # evaluate model performance on the validation set